# Análisis de sentimiento y agrupaciónes políticas en twitter
Lo primero que tenemos que hacer es extraer tweets desde la api que nos ofrece twitter, tenemos el curso de datacamp disponible [aquí](https://www.datacamp.com/courses/importing-data-in-python-part-2), *nos interesan los capítulos 2 y 3 especialmemte*
Voy a ir dejando un resumen de lo que contienen para hacerlo más fácil:
* El formato standard, aunque no el único, que nos devuelve una api es **JSON** (JavaScript Code Notation)
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQvI8lv3nmpUyybBdhcEjtq4sW5_w-AgSyQQfhC-F2D4WTg3GU4)


Sobre la API de Twitter, que es la que nos interesa tenemos la documentación [aquì](https://developer.twitter.com/en/docs.html)
Una **diferencia** importante que debemos conocer de la API de Twitter respecto de otras APIs es que la de Twitter requiere que tengamos una cuenta personal para realizar los requests, no podemos realizarlas de manera anónima.

Una vez tenemos cuenta, accedemos al siguiente [link](https://developer.twitter.com/en/apps).
Hacemos click en "Create an App" y seguimos los pasos.
Copio la respuesta más larga que tenéis que dar:
1. A School Project for EAE Business School
2. We plan to analyze tweets and users to build a network of the current political framework in Spain and the connections of their members.
3. No
4. Using the API, we import tweets and user data to Python to show Tables and Graphs about the political representation in Twitter.

Una vez rellenamos todo el formulario, nos envían un correo para que confirmemos nuestra cuenta, y nos toca esperar a que nos activen la cuenta de desarrollador.

Con la API de streaming de Twitter, podemos tener acceso al 1% de todos los tweets de la red si decidimos no filtrarlos (sample), u obtener todos los que cumplan un filtro (filter).  Para ello necesitamos el paquete de [**tweepy**](https://tweepy.readthedocs.io/en/3.7.0/index.html)

In [2]:
import tweepy,json
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
import sys
import networkx as nx

api_key = '812CaU2hXr9vz2OUjpXWDxooM' 
api_secret_key = 'NW0uyvtLWI6g96u7ImN5iJtm5Gkzv6mnguL3WOd3jXwOY5E6Da'
access_token = '1094207020109099008-kuGFfu8hC3RZTJVeZUjbEmMMJx73gD'
access_token_secret = 'kIZZic5s3JM4d7uVkYAxVWe0CExfLNyeMLS5rfo7fv68l'

auth = tweepy.OAuthHandler(api_key,api_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = API(auth)

Para importar el objeto SListener obtenermos el código de [este](https://github.com/SocialDataAnalytics-Winter2018/lab04/blob/master/slistener.py) enlace de Github proporcionado por *Datacamp*

In [3]:
class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')


    def on_data(self, data):
        if  'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return


    def on_status(self, status):
        self.output.write(status)
        self.counter += 1
        if self.counter >= 20000:
            self.output.close()
            self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            self.counter = 0
        return


    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return


    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return


    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        return 


    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return 

Hoy es día 15 de febrero de 2019, el presidente Pedro Sánchez ha anunciado la convocatoria de elecciones para el próximo 28 de febrero. Filtramos los tweets para encontrar los hashtags **#EleccionesGenerales** y **#28Abril**

Para obtener los datos de la API utilizaremos la *Streaming API* de twitter.  De este modo obtendremos tweets solo cuando estemos conectados a la API en tiempo real, no podemos obtener tweets del pasado.

Esta nos permite obtener tweets filtrando por *palabras clave*, *nombres de usuario* o *localizaciones*.

La [documentación](https://developer.twitter.com/en/docs/tweets/filter-realtime/overview) nos indica que, en la versión standard,  podemos filtrar por hasta un máximo de:
* 400 palabras clave
* 5000 nombres de usuario
* 25 localizaciones.

Además, debemos tener en cuenta la manera en la que queremos filtrar las palabras clave, para distinguir si queremos una frase exacta o palabras por separado.  Lo tenemos en el siguiente [link](https://developer.twitter.com/en/docs/tweets/rules-and-filtering/overview/standard-operators.html)

In [4]:
#Dividimos las cuentas políticas a seguir por partido político:
PP = ['@PPopular','@pablocasado_','@TeoGarciaEgea','@ALevySoler','@JavierMaroto','@DolorsMM','@MartaGlezVzqz','@jaimedeolano',
     '@JuanMa_Moreno','@FeijooGalicia']
Ciudadanos = ['@CiudadanosCs','@Albert_Rivera','@InesArrimadas','@GirautaOficial','@ignacioaguado',
              '@begonavillacis','@Tonicanto1','@lugaricano']
Vox = ['@vox_es','@Santi_ABASCAL','@ivanedlm','@Ortega_Smith','@monasterioR','@FSerranoCastro']
PSOE = ['@PSOE','@sanchezcastejon','@abalosmeco','@susanadiaz','@JosepBorrellF','@carmencalvo_',
        '@meritxell_batet','@Teresaribera','@mjmonteroc','@miqueliceta','@GFVara']
Podemos_IU_Compromis_EnMarea =['@ahorapodemos','@Pablo_Iglesias_','@VeraNoelia','@pnique','@Irene_Montero_','@ionebelarra',
                              '@GloriaElizo','@MayoralRafa','@TeresaRodr_','@MonederoJC','@RamonEspinar',
                               '@Julio_Rodr_','@agarzon','@monicaoltra']

Todos = PP+Ciudadanos+Vox+PSOE+Podemos_IU_Compromis_EnMarea

# Set up words to track
keywords_to_track = Todos

# Instantiate the SListener object 
listen = SListener(api)

# Instantiate the Stream object
stream = Stream(auth, listen)

# Begin collecting data
stream.filter(track = keywords_to_track)

KeyboardInterrupt: 

# Análisis de sentimiento

Una vez tenemos la muestra, el primer paso será realizar un **análisis de sentimiento** utilizando las palabras contenidas en los tweets.

Este método consiste en contar las palabras con connotación positiva y negativa en un texto, en este caso, tweet, para después ponerlas en proporción con el tweet entero.

La utilidad de este método se encuentra en cuantificar la reacción que produce en redes sociales un producto, empresa, ley o lo que es del interés de este trabajo: **políticos**.

Esta metodología produce un resultado numérico que va desde **-1** a **+1** según lo negativo o positivo que sea el tweet en su conjunto.

La librería que vamos a utilizar, llamada *VADER* es apropiada para textos cortos, distingue mayúsculas e incluye emojis 🤓
 



# Network Analysis